In [35]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pylab as plt
import seaborn as sns
%matplotlib inline
import time
from numpy.random import rand


In [36]:
from CCE_algorithm import CCE
# input (connectivity, max_iteration, target_center, check)
# output (n_center, center_id, label, n_cut)
from CCE_algorithm import connectivity_matrix
# input (X, sigma)
# output (similarity matrix)
from CCE_algorithm import enhanced_connectivity_matrix
# input (X, sigma, epsilon)
# output (enhanced matrix)|

from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import normalized_mutual_info_score


In [37]:
# data reading

penbased = pd.read_csv('./real_data/penbased.csv',header=None)
penbased_label = penbased.iloc[:,16]
penbased_data = penbased.iloc[:,:16]

In [38]:
# 0,2,4,6
penbased_even_data = penbased_data.iloc[list(map(lambda x: x in [0,2,4,6],penbased_label)),:]
penbased_even_label = penbased_label.iloc[list(map(lambda x: x in [0,2,4,6],penbased_label))]

In [39]:
# 1,3,5,7
penbased_odd_data = penbased_data.iloc[list(map(lambda x: x in [1,3,5,7],penbased_label)),:]
penbased_odd_label = penbased_label.iloc[list(map(lambda x: x in [1,3,5,7],penbased_label))]


In [40]:
seeds = pd.read_csv('./real_data/seeds.csv',header=None)
seeds.head()
seeds_label = seeds.iloc[:,7]
seeds_data = seeds.iloc[:,:7]

In [41]:
usps = pd.read_csv('./real_data/usps.csv',index_col=0)
usps.head()
usps_label = usps.iloc[:,64]
usps_data = usps.iloc[:,:64]

In [42]:
# 0,2,4,6
usps_even_data = usps_data.iloc[list(map(lambda x: x in [0,2,4,6],usps_label)),:]
usps_even_label = usps_label.iloc[list(map(lambda x: x in [0,2,4,6],usps_label))]

In [43]:
# 1,3,5,7
usps_odd_data = usps_data.iloc[list(map(lambda x: x in [1,3,5,7],usps_label)),:]
usps_odd_label = usps_label.iloc[list(map(lambda x: x in [1,3,5,7],usps_label))]

In [44]:
wdbc = pd.read_csv('./real_data/wdbc.csv',index_col=0,header=None)
wdbc.head()
wdbc_data = wdbc.iloc[1:,1:]
wdbc_label = wdbc.iloc[1:,0]
wdbc_label = list(map(lambda x : 1 if x=='M' else 0, list(wdbc_label.T)))

In [45]:
wine = pd.read_csv('./real_data/wine.csv',header=None)
wine.head()
wine_data = wine.iloc[:,1:]
wine_label = wine.iloc[:,0]

In [46]:
iris = pd.read_csv('./real_data/iris.csv',index_col=0)
iris.head()
iris_data = iris.iloc[:,1:]
iris_label = iris.iloc[:,0]

In [59]:
def connectivity_matrix_simple(X, sigma):
    """ Calculate ergodic connectivity with gaussian kernel
    
    Args :
        X : data (array)
        sigma : standard deviation (numeric)
        
    Returns :
        ergodic connectivity matrix (array)
    """
    
    n, p = X.shape
    D = -2 * np.dot(X, X.T) + np.tile(np.sum(X ** 2, 1), 
                                      (n, 1)).T + np.tile(np.sum(X ** 2, 1), (n, 1))
    S = np.exp(-D / (2 * sigma ** 2))
    return S

In [60]:
S_iris = connectivity_matrix_simple(iris_data, 0.2)
S_wine = connectivity_matrix_simple(wine_data, 32)
S_usps_odd = connectivity_matrix_simple(usps_odd_data, 8)
S_usps_even = connectivity_matrix_simple(usps_even_data, 8)
S_penbased_odd = connectivity_matrix_simple(penbased_odd_data, 25)
S_penbased_even = connectivity_matrix_simple(penbased_even_data, 25)
S_seeds = connectivity_matrix_simple(seeds_data, 0.4)

In [61]:
df_describe = pd.DataFrame(S_iris.reshape(-1,1))
df_describe.describe()

,0
count,2.250000e+04
mean,2.360992e-02
std,1.123653e-01
min,3.021430e-273
25%,4.207959e-82
50%,3.328603e-30
75%,1.760346e-06
max,1.000000e+00


In [62]:
df_describe = pd.DataFrame(S_wine.reshape(-1,1))
df_describe.describe()

,0
count,3.168400e+04
mean,7.650652e-02
std,2.123531e-01
min,0.000000e+00
25%,1.185439e-60
50%,2.218300e-17
75%,4.329043e-04
max,1.000000e+00


In [63]:
df_describe = pd.DataFrame(S_usps_even.reshape(-1,1))
df_describe.describe()

,0
count,5.004169e+06
mean,2.662227e-03
std,2.548610e-02
min,1.753505e-19
25%,2.461697e-10
50%,1.333582e-08
75%,1.529415e-06
max,1.000000e+00


In [64]:
df_describe = pd.DataFrame(S_usps_odd.reshape(-1,1))
df_describe.describe()

,0
count,5.139289e+06
mean,1.501703e-03
std,2.330182e-02
min,5.680842e-21
25%,3.872793e-10
50%,1.808605e-08
75%,1.425572e-06
max,1.000000e+00


In [65]:
df_describe = pd.DataFrame(S_penbased_odd.reshape(-1,1))
df_describe.describe()

,0
count,1.931602e+07
mean,1.831252e-02
std,7.625066e-02
min,2.336731e-30
25%,4.176281e-12
50%,2.272046e-08
75%,7.497169e-05
max,1.000000e+00


In [66]:
df_describe = pd.DataFrame(S_penbased_even.reshape(-1,1))
df_describe.describe()

,0
count,2.013317e+07
mean,1.879592e-02
std,7.691708e-02
min,1.467445e-32
25%,2.985019e-13
50%,1.109674e-09
75%,1.091252e-05
max,1.000000e+00


In [67]:
df_describe = pd.DataFrame(S_seeds.reshape(-1,1))
df_describe.describe()

,0
count,4.410000e+04
mean,1.867976e-02
std,1.005574e-01
min,8.569681e-194
25%,1.646741e-52
50%,7.964601e-23
75%,2.332477e-08
max,1.000000e+00


In [ ]:
S_mnist_fashion = connectivity_matrix(iris_data, sigma)

S_olivetti_face = connectivity_matrix(iris_data, sigma)
S_arrhythmia = connectivity_matrix(iris_data, sigma)

In [47]:
def Experiment(X,y,sigma,epsilon=0,max_it=1000):
    start_time = time.time()
    n_cluster = len(set(list(y)))
    if epsilon ==0 :
        S = connectivity_matrix(X,sigma)
    else :
        S = enhanced_connectivity_matrix(X,sigma,epsilon)
    result = CCE(S,max_it,n_cluster,10)
    runtime = (time.time() - start_time)
    iteration = len(result[0])
    ARI = adjusted_rand_score(result[2], np.array(y))
    NMI = normalized_mutual_info_score(result[2], np.array(y))
    return np.array([sigma, epsilon ,iteration, ARI, NMI, runtime])

In [48]:
Experiment(iris_data,iris_label,0.5)

order of connectivity matrix = 1
number of cluster center = 148
...
less then 3 cluster center
order 10
[7, 78, 147]


C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


array([ 0.5       ,  0.        , 10.        ,  0.75760573,  0.77435074,
        0.02789664])

In [17]:
sigma_list = [0.1,0.3,1,1.5,2,3,5,10]
epsilon_list = [0,1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
data_list = [penbased_even_data,penbased_odd_data,iris_data,wine_data,seeds_data,
             wdbc_data,usps_even_data,usps_odd_data]
label_list = [penbased_even_label,penbased_odd_label,iris_label,wine_label,
              seeds_label,wdbc_label,usps_even_label,usps_odd_label]

In [22]:
sigma_list = [0.2]
for sigma in sigma_list:
    print(Experiment(iris_data,iris_label,sigma,0,2000))

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 1
number of cluster center = 148
...
order of connectivity matrix = 11
number of cluster center = 36
...
order of connectivity matrix = 21
number of cluster center = 18
...
order of connectivity matrix = 31
number of cluster center = 16
...
order of connectivity matrix = 41
number of cluster center = 15
...
order of connectivity matrix = 51
number of cluster center = 12
...
order of connectivity matrix = 61
number of cluster center = 10
...
order of connectivity matrix = 71
number of cluster center = 9
...
order of connectivity matrix = 81
number of cluster center = 9
...
order of connectivity matrix = 91
number of cluster center = 9
...
order of connectivity matrix = 101
number of cluster center = 9
...
order of connectivity matrix = 111
number of cluster center = 9
...
order of connectivity matrix = 121
number of cluster center = 9
...
order of connectivity matrix = 131
number of cluster center = 9
...
order of connectivity matrix = 141
number of cluste

order of connectivity matrix = 1291
number of cluster center = 5
...
order of connectivity matrix = 1301
number of cluster center = 5
...
order of connectivity matrix = 1311
number of cluster center = 5
...
order of connectivity matrix = 1321
number of cluster center = 4
...
order of connectivity matrix = 1331
number of cluster center = 4
...
order of connectivity matrix = 1341
number of cluster center = 4
...
order of connectivity matrix = 1351
number of cluster center = 4
...
order of connectivity matrix = 1361
number of cluster center = 4
...
order of connectivity matrix = 1371
number of cluster center = 4
...
order of connectivity matrix = 1381
number of cluster center = 4
...
order of connectivity matrix = 1391
number of cluster center = 4
...
order of connectivity matrix = 1401
number of cluster center = 4
...
order of connectivity matrix = 1411
number of cluster center = 4
...
order of connectivity matrix = 1421
number of cluster center = 4
...
order of connectivity matrix = 143

In [23]:
# epsilon_list = [0,1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
for epsilon in epsilon_list:
    print(Experiment(iris_data,iris_label,0.2,epsilon,2000))

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 1
number of cluster center = 148
...
order of connectivity matrix = 11
number of cluster center = 36
...
order of connectivity matrix = 21
number of cluster center = 18
...
order of connectivity matrix = 31
number of cluster center = 16
...
order of connectivity matrix = 41
number of cluster center = 15
...
order of connectivity matrix = 51
number of cluster center = 12
...
order of connectivity matrix = 61
number of cluster center = 10
...
order of connectivity matrix = 71
number of cluster center = 9
...
order of connectivity matrix = 81
number of cluster center = 9
...
order of connectivity matrix = 91
number of cluster center = 9
...
order of connectivity matrix = 101
number of cluster center = 9
...
order of connectivity matrix = 111
number of cluster center = 9
...
order of connectivity matrix = 121
number of cluster center = 9
...
order of connectivity matrix = 131
number of cluster center = 9
...
order of connectivity matrix = 141
number of cluste

order of connectivity matrix = 1301
number of cluster center = 5
...
order of connectivity matrix = 1311
number of cluster center = 5
...
order of connectivity matrix = 1321
number of cluster center = 4
...
order of connectivity matrix = 1331
number of cluster center = 4
...
order of connectivity matrix = 1341
number of cluster center = 4
...
order of connectivity matrix = 1351
number of cluster center = 4
...
order of connectivity matrix = 1361
number of cluster center = 4
...
order of connectivity matrix = 1371
number of cluster center = 4
...
order of connectivity matrix = 1381
number of cluster center = 4
...
order of connectivity matrix = 1391
number of cluster center = 4
...
order of connectivity matrix = 1401
number of cluster center = 4
...
order of connectivity matrix = 1411
number of cluster center = 4
...
order of connectivity matrix = 1421
number of cluster center = 4
...
order of connectivity matrix = 1431
number of cluster center = 4
...
order of connectivity matrix = 144

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])



number of cluster center = 9
...
order of connectivity matrix = 131
number of cluster center = 9
...
order of connectivity matrix = 141
number of cluster center = 9
...
order of connectivity matrix = 151
number of cluster center = 9
...
order of connectivity matrix = 161
number of cluster center = 9
...
order of connectivity matrix = 171
number of cluster center = 9
...
order of connectivity matrix = 181
number of cluster center = 9
...
order of connectivity matrix = 191
number of cluster center = 8
...
order of connectivity matrix = 201
number of cluster center = 8
...
order of connectivity matrix = 211
number of cluster center = 8
...
order of connectivity matrix = 221
number of cluster center = 8
...
order of connectivity matrix = 231
number of cluster center = 8
...
order of connectivity matrix = 241
number of cluster center = 8
...
order of connectivity matrix = 251
number of cluster center = 8
...
order of connectivity matrix = 261
number of cluster center = 8
...
order of conne

...

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])



number of cluster center = 9
...
order of connectivity matrix = 141
number of cluster center = 9
...
order of connectivity matrix = 151
number of cluster center = 9
...
order of connectivity matrix = 161
number of cluster center = 8
...
order of connectivity matrix = 171
number of cluster center = 8
...
order of connectivity matrix = 181
number of cluster center = 8
...
order of connectivity matrix = 191
number of cluster center = 8
...
order of connectivity matrix = 201
number of cluster center = 8
...
order of connectivity matrix = 211
number of cluster center = 7
...
order of connectivity matrix = 221
number of cluster center = 6
...
order of connectivity matrix = 231
number of cluster center = 6
...
order of connectivity matrix = 241
number of cluster center = 6
...
order of connectivity matrix = 251
number of cluster center = 6
...
order of connectivity matrix = 261
number of cluster center = 5
...
order of connectivity matrix = 271
number of cluster center = 5
...
order of conne

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 31
number of cluster center = 16
...
order of connectivity matrix = 41
number of cluster center = 15
...
order of connectivity matrix = 51
number of cluster center = 13
...
order of connectivity matrix = 61
number of cluster center = 9
...
order of connectivity matrix = 71
number of cluster center = 9
...
order of connectivity matrix = 81
number of cluster center = 9
...
order of connectivity matrix = 91
number of cluster center = 9
...
order of connectivity matrix = 101
number of cluster center = 9
...
order of connectivity matrix = 111
number of cluster center = 9
...
order of connectivity matrix = 121
number of cluster center = 9
...
order of connectivity matrix = 131
number of cluster center = 9
...
order of connectivity matrix = 141
number of cluster center = 8
...
order of connectivity matrix = 151
number of cluster center = 8
...
order of connectivity matrix = 161
number of cluster center = 7
...
order of connectivity matrix = 171
number of cluster

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


In [100]:
stack = np.zeros(5)
for i in range(len(data_list)):
    for sigma in sigma_list:
        tmp = Experiment(data_list[i],label_list[i],sigma,0)
        stack = np.vstack([stack, tmp])

order of similarity matrix = 1
number of cluster center = 4487
...
order of similarity matrix = 11
number of cluster center = 4487
...
order of similarity matrix = 21
number of cluster center = 4487
...
order of similarity matrix = 31
number of cluster center = 4487
...
order of similarity matrix = 41
number of cluster center = 4487
...
order of similarity matrix = 51
number of cluster center = 4487
...


KeyboardInterrupt: 

In [102]:
# penbased_even
sigma_list = [25,30,35,40,45,50]
for sigma in sigma_list:
    print(Experiment(penbased_even_data,penbased_even_label,sigma,0))


order of similarity matrix = 1
number of cluster center = 4487
...
order of similarity matrix = 11
number of cluster center = 25
...
order of similarity matrix = 21
number of cluster center = 15
...
order of similarity matrix = 31
number of cluster center = 12
...
order of similarity matrix = 41
number of cluster center = 12
...
order of similarity matrix = 51
number of cluster center = 11
...
order of similarity matrix = 61
number of cluster center = 11
...
order of similarity matrix = 71
number of cluster center = 10
...
order of similarity matrix = 81
number of cluster center = 9
...
order of similarity matrix = 91
number of cluster center = 9
...
order of similarity matrix = 101
number of cluster center = 9
...
order of similarity matrix = 111
number of cluster center = 9
...
order of similarity matrix = 121
number of cluster center = 8
...
order of similarity matrix = 131
number of cluster center = 8
...
order of similarity matrix = 141
number of cluster center = 8
...
order of si

In [103]:
sigma_list = [25,30,35,40,45,50]
for sigma in sigma_list:
    print(Experiment(penbased_odd_data,penbased_odd_label,sigma,0))


order of similarity matrix = 1
number of cluster center = 3253
...
order of similarity matrix = 11
number of cluster center = 16
...
order of similarity matrix = 21
number of cluster center = 13
...
order of similarity matrix = 31
number of cluster center = 11
...
order of similarity matrix = 41
number of cluster center = 10
...
order of similarity matrix = 51
number of cluster center = 10
...
order of similarity matrix = 61
number of cluster center = 10
...
order of similarity matrix = 71
number of cluster center = 9
...
order of similarity matrix = 81
number of cluster center = 8
...
order of similarity matrix = 91
number of cluster center = 8
...
order of similarity matrix = 101
number of cluster center = 7
...
order of similarity matrix = 111
number of cluster center = 7
...
order of similarity matrix = 121
number of cluster center = 7
...
order of similarity matrix = 131
number of cluster center = 6
...
order of similarity matrix = 141
number of cluster center = 6
...
order of sim

In [107]:
sigma_list = [35,40,45,50]
for sigma in sigma_list:
    print(Experiment(penbased_odd_data,penbased_odd_label,sigma,0))

order of similarity matrix = 1
number of cluster center = 3252
...
order of similarity matrix = 11
number of cluster center = 6
...
order of similarity matrix = 21
number of cluster center = 4
...
less then 3 cluster center
order 23
[1682, 2460, 2969]
[35.          0.         23.          0.48796896  0.58278212]
order of similarity matrix = 1
number of cluster center = 3252
...
less then 3 cluster center
order 8
[1682, 2460, 2969]
[40.          0.          8.          0.48815419  0.58168097]
order of similarity matrix = 1
number of cluster center = 3252
...
less then 3 cluster center
order 5
[626, 1682, 2969]
[45.          0.          5.          0.48815419  0.58168097]
order of similarity matrix = 1
number of cluster center = 3252
...
less then 3 cluster center
order 4
[626, 1682, 2969]
[50.          0.          4.          0.48565488  0.58005624]


In [89]:
epsilon_list = [0,1e-6,5e-4]
for epsilon in epsilon_list:
    print(np.round(Experiment(penbased_odd_data,penbased_odd_label,25,epsilon,3000),4))

order of similarity matrix = 1
number of cluster center = 4395
...
order of similarity matrix = 501
number of cluster center = 5
...
order of similarity matrix = 1001
number of cluster center = 5
...
order of similarity matrix = 1501
number of cluster center = 5
...
less then 4 cluster center
order 1625
[14, 3008, 3318, 3784]
[2.500e+01 0.000e+00 1.625e+03 1.062e-01 2.718e-01]
order of similarity matrix = 1
number of cluster center = 4395
...
order of similarity matrix = 501
number of cluster center = 5
...
order of similarity matrix = 1001
number of cluster center = 5
...
less then 4 cluster center
order 1051
[14, 1983, 3008, 3318]
[2.500e+01 0.000e+00 1.051e+03 2.792e-01 4.285e-01]
order of similarity matrix = 1
number of cluster center = 4395
...
less then 4 cluster center
order 32
[14, 2045, 3318, 3692]
[2.500e+01 5.000e-04 3.200e+01 4.417e-01 5.349e-01]


In [24]:
epsilon_list = [0,1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
for epsilon in epsilon_list:
    print(np.round(Experiment(penbased_odd_data,penbased_odd_label,25,epsilon,3000),4))

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 1
number of cluster center = 4395
...
order of connectivity matrix = 11
number of cluster center = 20
...
order of connectivity matrix = 21
number of cluster center = 16
...
order of connectivity matrix = 31
number of cluster center = 13
...
order of connectivity matrix = 41
number of cluster center = 11
...
order of connectivity matrix = 51
number of cluster center = 10
...
order of connectivity matrix = 61
number of cluster center = 10
...
order of connectivity matrix = 71
number of cluster center = 9
...
order of connectivity matrix = 81
number of cluster center = 8
...
order of connectivity matrix = 91
number of cluster center = 8
...
order of connectivity matrix = 101
number of cluster center = 7
...
order of connectivity matrix = 111
number of cluster center = 7
...
order of connectivity matrix = 121
number of cluster center = 6
...
order of connectivity matrix = 131
number of cluster center = 6
...
order of connectivity matrix = 141
number of clust

order of connectivity matrix = 1211
number of cluster center = 5
...
order of connectivity matrix = 1221
number of cluster center = 5
...
order of connectivity matrix = 1231
number of cluster center = 5
...
order of connectivity matrix = 1241
number of cluster center = 5
...
order of connectivity matrix = 1251
number of cluster center = 5
...
order of connectivity matrix = 1261
number of cluster center = 5
...
order of connectivity matrix = 1271
number of cluster center = 5
...
order of connectivity matrix = 1281
number of cluster center = 5
...
order of connectivity matrix = 1291
number of cluster center = 5
...
order of connectivity matrix = 1301
number of cluster center = 5
...
order of connectivity matrix = 1311
number of cluster center = 5
...
order of connectivity matrix = 1321
number of cluster center = 5
...
order of connectivity matrix = 1331
number of cluster center = 5
...
order of connectivity matrix = 1341
number of cluster center = 5
...
order of connectivity matrix = 135

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 1
number of cluster center = 4395
...
order of connectivity matrix = 11
number of cluster center = 20
...
order of connectivity matrix = 21
number of cluster center = 16
...
order of connectivity matrix = 31
number of cluster center = 13
...
order of connectivity matrix = 41
number of cluster center = 11
...
order of connectivity matrix = 51
number of cluster center = 10
...
order of connectivity matrix = 61
number of cluster center = 10
...
order of connectivity matrix = 71
number of cluster center = 9
...
order of connectivity matrix = 81
number of cluster center = 8
...
order of connectivity matrix = 91
number of cluster center = 7
...
order of connectivity matrix = 101
number of cluster center = 7
...
order of connectivity matrix = 111
number of cluster center = 7
...
order of connectivity matrix = 121
number of cluster center = 6
...
order of connectivity matrix = 131
number of cluster center = 6
...
order of connectivity matrix = 141
number of clust

...

order of connectivity matrix = 1
number of cluster center = 4395
...
order of connectivity matrix = 11
number of cluster center = 18
...
order of connectivity matrix = 21
number of cluster center = 13
...
order of connectivity matrix = 31
number of cluster center = 9
...
order of connectivity matrix = 41
number of cluster center = 6
...
order of connectivity matrix = 51
number of cluster center = 6
...
order of connectivity matrix = 61
number of cluster center = 5
...
less then 4 cluster center
order 67
[14, 1983, 2589, 3318]
[25.      0.     67.      0.4452  0.5377 65.0217]


C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 1
number of cluster center = 4395
...
order of connectivity matrix = 11
number of cluster center = 19
...
order of connectivity matrix = 21
number of cluster center = 9
...
order of connectivity matrix = 31
number of cluster center = 6
...
order of connectivity matrix = 41
number of cluster center = 6
...
order of connectivity matrix = 51
number of cluster center = 6
...
less then 4 cluster center
order 59
[14, 1983, 2589, 3318]
[2.50000e+01 1.00000e-04 5.90000e+01 4.45500e-01 5.37900e-01 5.74881e+01]


C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 1
number of cluster center = 4395
...
order of connectivity matrix = 11
number of cluster center = 8
...
order of connectivity matrix = 21
number of cluster center = 7
...
order of connectivity matrix = 31
number of cluster center = 5
...
less then 4 cluster center
order 32
[14, 2045, 3318, 3692]
[2.50000e+01 5.00000e-04 3.20000e+01 4.44300e-01 5.36300e-01 3.13103e+01]


In [25]:
epsilon_list = [0,1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
pen_even_result = np.zeros(5)
for epsilon in epsilon_list:
    tmp = Experiment(penbased_even_data,penbased_even_label,40,epsilon)
    print(tmp)
    pen_even_result = np.vstack((pen_even_result,tmp))
pen_even_result

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 1
number of cluster center = 4487
...
order of connectivity matrix = 11
number of cluster center = 7
...
order of connectivity matrix = 21
number of cluster center = 5
...
less then 4 cluster center
order 28
[285, 649, 905, 3227]
[40.          0.         28.          0.92198406  0.90655663 28.98511648]


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 5 and the array at index 1 has size 6

In [117]:
np.round(pen_even_result,3)

array([[0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00],
       [4.00e+01, 0.00e+00, 2.80e+01, 8.40e-01, 8.49e-01],
       [4.00e+01, 0.00e+00, 2.80e+01, 8.38e-01, 8.48e-01],
       [4.00e+01, 0.00e+00, 2.80e+01, 8.20e-01, 8.38e-01],
       [4.00e+01, 0.00e+00, 2.70e+01, 7.95e-01, 8.25e-01],
       [4.00e+01, 0.00e+00, 2.10e+01, 5.33e-01, 6.97e-01],
       [4.00e+01, 0.00e+00, 1.30e+01, 2.78e-01, 4.80e-01],
       [4.00e+01, 0.00e+00, 6.00e+00, 0.00e+00, 1.00e-03]])

In [123]:
sigma_list = [0.4,0.5,0.8,1,1.5]
for sigma in sigma_list:
    print(np.round(Experiment(iris_data,iris_label,sigma,0),4))

order of similarity matrix = 1
number of cluster center = 148
...
order of similarity matrix = 11
number of cluster center = 7
...
less then 3 cluster center
order 19
[7, 78, 147]
[ 0.4     0.     19.      0.7455  0.798 ]
order of similarity matrix = 1
number of cluster center = 148
...
less then 3 cluster center
order 10
[7, 78, 147]
[ 0.5     0.     10.      0.7592  0.8057]
order of similarity matrix = 1
number of cluster center = 148
...
less then 3 cluster center
order 5
[7, 126]
[0.8    0.     5.     0.5681 0.7337]
order of similarity matrix = 1
number of cluster center = 148
...
less then 3 cluster center
order 3
[7, 126]
[1.     0.     3.     0.5681 0.7337]
order of similarity matrix = 1
number of cluster center = 148
...
less then 3 cluster center
order 2
[7, 126, 147]
[1.5    0.     2.     0.7155 0.7614]


In [23]:
epsilon_list = [0,1e-6,5e-6,1e-5,5e-5,1e-4,5e-4,1e-4,5e-3]
for epsilon in epsilon_list:
    print(np.round(Experiment(iris_data,iris_label,0.4,epsilon),4))

order of similarity matrix = 1
number of cluster center = 148
...
order of similarity matrix = 11
number of cluster center = 7
...
less then 3 cluster center
order 19
[7, 78, 147]
[ 0.4     0.     19.      0.7455  0.798 ]
order of similarity matrix = 1
number of cluster center = 148
...
order of similarity matrix = 11
number of cluster center = 7
...
less then 3 cluster center
order 19
[7, 78, 147]
[ 0.4     0.     19.      0.7455  0.798 ]
order of similarity matrix = 1
number of cluster center = 148
...
order of similarity matrix = 11
number of cluster center = 7
...
less then 3 cluster center
order 19
[7, 78, 147]
[ 0.4     0.     19.      0.7455  0.798 ]
order of similarity matrix = 1
number of cluster center = 148
...
order of similarity matrix = 11
number of cluster center = 7
...
less then 3 cluster center
order 19
[7, 78, 147]
[ 0.4     0.     19.      0.7455  0.798 ]
order of similarity matrix = 1
number of cluster center = 148
...
order of similarity matrix = 11
number of clus

In [131]:
sigma_list = [0.4,0.5,0.8,1,1.5]
for sigma in sigma_list:
    print(np.round(Experiment(seeds_data,seeds_label,sigma,0),4))


order of similarity matrix = 1
number of cluster center = 210
...
order of similarity matrix = 11
number of cluster center = 43
...
order of similarity matrix = 21
number of cluster center = 24
...
order of similarity matrix = 31
number of cluster center = 14
...
order of similarity matrix = 41
number of cluster center = 12
...
order of similarity matrix = 51
number of cluster center = 11
...
order of similarity matrix = 61
number of cluster center = 9
...
order of similarity matrix = 71
number of cluster center = 8
...
order of similarity matrix = 81
number of cluster center = 8
...
order of similarity matrix = 91
number of cluster center = 8
...
order of similarity matrix = 101
number of cluster center = 8
...
order of similarity matrix = 111
number of cluster center = 7
...
order of similarity matrix = 121
number of cluster center = 7
...
order of similarity matrix = 131
number of cluster center = 7
...
order of similarity matrix = 141
number of cluster center = 7
...
order of simil

In [31]:
epsilon_list = [0,1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
for epsilon in epsilon_list:
    print(np.round(Experiment(seeds_data,seeds_label,0.4,epsilon,3000),4))

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 1
number of cluster center = 210
...
order of connectivity matrix = 11
number of cluster center = 43
...
order of connectivity matrix = 21
number of cluster center = 24
...
order of connectivity matrix = 31
number of cluster center = 14
...
order of connectivity matrix = 41
number of cluster center = 12
...
order of connectivity matrix = 51
number of cluster center = 11
...
order of connectivity matrix = 61
number of cluster center = 9
...
order of connectivity matrix = 71
number of cluster center = 8
...
order of connectivity matrix = 81
number of cluster center = 8
...
order of connectivity matrix = 91
number of cluster center = 8
...
order of connectivity matrix = 101
number of cluster center = 8
...
order of connectivity matrix = 111
number of cluster center = 7
...
order of connectivity matrix = 121
number of cluster center = 7
...
order of connectivity matrix = 131
number of cluster center = 7
...
order of connectivity matrix = 141
number of cluster

order of connectivity matrix = 1281
number of cluster center = 4
...
order of connectivity matrix = 1291
number of cluster center = 4
...
order of connectivity matrix = 1301
number of cluster center = 4
...
order of connectivity matrix = 1311
number of cluster center = 4
...
order of connectivity matrix = 1321
number of cluster center = 4
...
order of connectivity matrix = 1331
number of cluster center = 4
...
order of connectivity matrix = 1341
number of cluster center = 4
...
order of connectivity matrix = 1351
number of cluster center = 4
...
order of connectivity matrix = 1361
number of cluster center = 4
...
order of connectivity matrix = 1371
number of cluster center = 4
...
order of connectivity matrix = 1381
number of cluster center = 4
...
order of connectivity matrix = 1391
number of cluster center = 4
...
order of connectivity matrix = 1401
number of cluster center = 4
...
order of connectivity matrix = 1411
number of cluster center = 4
...
order of connectivity matrix = 142

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])



number of cluster center = 14
...
order of connectivity matrix = 41
number of cluster center = 12
...
order of connectivity matrix = 51
number of cluster center = 11
...
order of connectivity matrix = 61
number of cluster center = 9
...
order of connectivity matrix = 71
number of cluster center = 8
...
order of connectivity matrix = 81
number of cluster center = 8
...
order of connectivity matrix = 91
number of cluster center = 8
...
order of connectivity matrix = 101
number of cluster center = 8
...
order of connectivity matrix = 111
number of cluster center = 7
...
order of connectivity matrix = 121
number of cluster center = 7
...
order of connectivity matrix = 131
number of cluster center = 7
...
order of connectivity matrix = 141
number of cluster center = 7
...
order of connectivity matrix = 151
number of cluster center = 7
...
order of connectivity matrix = 161
number of cluster center = 7
...
order of connectivity matrix = 171
number of cluster center = 6
...
order of connecti

...

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 131
number of cluster center = 6
...
order of connectivity matrix = 141
number of cluster center = 6
...
order of connectivity matrix = 151
number of cluster center = 6
...
order of connectivity matrix = 161
number of cluster center = 5
...
order of connectivity matrix = 171
number of cluster center = 5
...
order of connectivity matrix = 181
number of cluster center = 5
...
order of connectivity matrix = 191
number of cluster center = 5
...
order of connectivity matrix = 201
number of cluster center = 5
...
order of connectivity matrix = 211
number of cluster center = 5
...
order of connectivity matrix = 221
number of cluster center = 5
...
order of connectivity matrix = 231
number of cluster center = 5
...
order of connectivity matrix = 241
number of cluster center = 5
...
order of connectivity matrix = 251
number of cluster center = 5
...
order of connectivity matrix = 261
number of cluster center = 5
...
order of connectivity matrix = 271
number of clu

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])



number of cluster center = 43
...
order of connectivity matrix = 21
number of cluster center = 24
...
order of connectivity matrix = 31
number of cluster center = 14
...
order of connectivity matrix = 41
number of cluster center = 12
...
order of connectivity matrix = 51
number of cluster center = 10
...
order of connectivity matrix = 61
number of cluster center = 8
...
order of connectivity matrix = 71
number of cluster center = 8
...
order of connectivity matrix = 81
number of cluster center = 8
...
order of connectivity matrix = 91
number of cluster center = 7
...
order of connectivity matrix = 101
number of cluster center = 7
...
order of connectivity matrix = 111
number of cluster center = 6
...
order of connectivity matrix = 121
number of cluster center = 5
...
order of connectivity matrix = 131
number of cluster center = 5
...
order of connectivity matrix = 141
number of cluster center = 5
...
order of connectivity matrix = 151
number of cluster center = 5
...
order of connecti

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


In [83]:
sigma_list = [8]
for sigma in sigma_list:
    print(np.round(Experiment(usps_odd_data,usps_odd_label,sigma,0,10000),4))

order of similarity matrix = 1
number of cluster center = 1701
...
order of similarity matrix = 501
number of cluster center = 5
...
less then 3 cluster center
order 574
[729, 927, 1613]
[8.000e+00 0.000e+00 5.740e+02 5.336e-01 6.878e-01]


In [29]:
#epsilon_list = [1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
epsilon_list = [0,1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
for epsilon in epsilon_list:
    print(np.round(Experiment(usps_odd_data,usps_odd_label,8,epsilon,4000),4))

order of connectivity matrix = 1
number of cluster center = 2267
...


C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 11
number of cluster center = 426
...
order of connectivity matrix = 21
number of cluster center = 189
...
order of connectivity matrix = 31
number of cluster center = 119
...
order of connectivity matrix = 41
number of cluster center = 82
...
order of connectivity matrix = 51
number of cluster center = 65
...
order of connectivity matrix = 61
number of cluster center = 56
...
order of connectivity matrix = 71
number of cluster center = 48
...
order of connectivity matrix = 81
number of cluster center = 40
...
order of connectivity matrix = 91
number of cluster center = 35
...
order of connectivity matrix = 101
number of cluster center = 34
...
order of connectivity matrix = 111
number of cluster center = 28
...
order of connectivity matrix = 121
number of cluster center = 27
...
order of connectivity matrix = 131
number of cluster center = 24
...
order of connectivity matrix = 141
number of cluster center = 23
...
order of connectivity matrix = 151
numbe

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 11
number of cluster center = 426
...
order of connectivity matrix = 21
number of cluster center = 189
...
order of connectivity matrix = 31
number of cluster center = 118
...
order of connectivity matrix = 41
number of cluster center = 83
...
order of connectivity matrix = 51
number of cluster center = 63
...
order of connectivity matrix = 61
number of cluster center = 56
...
order of connectivity matrix = 71
number of cluster center = 45
...
order of connectivity matrix = 81
number of cluster center = 39
...
order of connectivity matrix = 91
number of cluster center = 35
...
order of connectivity matrix = 101
number of cluster center = 33
...
order of connectivity matrix = 111
number of cluster center = 27
...
order of connectivity matrix = 121
number of cluster center = 27
...
order of connectivity matrix = 131
number of cluster center = 24
...
order of connectivity matrix = 141
number of cluster center = 23
...
order of connectivity matrix = 151
numbe

...

order of connectivity matrix = 11
number of cluster center = 414
...
order of connectivity matrix = 21
number of cluster center = 172
...
order of connectivity matrix = 31
number of cluster center = 76
...
order of connectivity matrix = 41
number of cluster center = 37
...
order of connectivity matrix = 51
number of cluster center = 17
...
less then 4 cluster center
order 60
[971, 1234, 1804, 2153]
[ 8.      0.     60.      0.5416  0.6932 11.6862]
order of connectivity matrix = 1
number of cluster center = 2267
...


C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 11
number of cluster center = 397
...
order of connectivity matrix = 21
number of cluster center = 103
...
order of connectivity matrix = 31
number of cluster center = 6
...
order of connectivity matrix = 41
number of cluster center = 4
...
less then 4 cluster center
order 41
[1234, 1804, 2153, 2185]
[8.0000e+00 1.0000e-04 4.1000e+01 5.2900e-01 6.8670e-01 8.2965e+00]
order of connectivity matrix = 1
number of cluster center = 2267
...


C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 11
number of cluster center = 7
...
less then 4 cluster center
order 13
[1234, 1804, 2153, 2185]
[8.000e+00 5.000e-04 1.300e+01 2.692e-01 4.802e-01 3.182e+00]


In [30]:
#epsilon_list = [1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
epsilon_list = [0,1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
for epsilon in epsilon_list:
    print(np.round(Experiment(usps_even_data,usps_even_label,8,epsilon,4000),4))


C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 1
number of cluster center = 2237
...
order of connectivity matrix = 11
number of cluster center = 257
...
order of connectivity matrix = 21
number of cluster center = 120
...
order of connectivity matrix = 31
number of cluster center = 85
...
order of connectivity matrix = 41
number of cluster center = 56
...
order of connectivity matrix = 51
number of cluster center = 48
...
order of connectivity matrix = 61
number of cluster center = 42
...
order of connectivity matrix = 71
number of cluster center = 33
...
order of connectivity matrix = 81
number of cluster center = 29
...
order of connectivity matrix = 91
number of cluster center = 27
...
order of connectivity matrix = 101
number of cluster center = 24
...
order of connectivity matrix = 111
number of cluster center = 21
...
order of connectivity matrix = 121
number of cluster center = 19
...
order of connectivity matrix = 131
number of cluster center = 19
...
order of connectivity matrix = 141
number

order of connectivity matrix = 1201
number of cluster center = 6
...
order of connectivity matrix = 1211
number of cluster center = 6
...
order of connectivity matrix = 1221
number of cluster center = 6
...
order of connectivity matrix = 1231
number of cluster center = 6
...
order of connectivity matrix = 1241
number of cluster center = 6
...
order of connectivity matrix = 1251
number of cluster center = 6
...
order of connectivity matrix = 1261
number of cluster center = 6
...
order of connectivity matrix = 1271
number of cluster center = 6
...
order of connectivity matrix = 1281
number of cluster center = 5
...
order of connectivity matrix = 1291
number of cluster center = 5
...
order of connectivity matrix = 1301
number of cluster center = 5
...
order of connectivity matrix = 1311
number of cluster center = 5
...
order of connectivity matrix = 1321
number of cluster center = 5
...
order of connectivity matrix = 1331
number of cluster center = 5
...
order of connectivity matrix = 134

order of connectivity matrix = 2391
number of cluster center = 5
...
order of connectivity matrix = 2401
number of cluster center = 5
...
order of connectivity matrix = 2411
number of cluster center = 5
...
order of connectivity matrix = 2421
number of cluster center = 5
...
order of connectivity matrix = 2431
number of cluster center = 5
...
order of connectivity matrix = 2441
number of cluster center = 5
...
order of connectivity matrix = 2451
number of cluster center = 5
...
order of connectivity matrix = 2461
number of cluster center = 5
...
order of connectivity matrix = 2471
number of cluster center = 5
...
order of connectivity matrix = 2481
number of cluster center = 5
...
order of connectivity matrix = 2491
number of cluster center = 5
...
order of connectivity matrix = 2501
number of cluster center = 5
...
order of connectivity matrix = 2511
number of cluster center = 5
...
order of connectivity matrix = 2521
number of cluster center = 5
...
order of connectivity matrix = 253

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


...

order of connectivity matrix = 1
number of cluster center = 2237
...
order of connectivity matrix = 11
number of cluster center = 257
...
order of connectivity matrix = 21
number of cluster center = 108
...
order of connectivity matrix = 31
number of cluster center = 56
...
order of connectivity matrix = 41
number of cluster center = 33
...
order of connectivity matrix = 51
number of cluster center = 13
...
less then 4 cluster center
order 58
[144, 1392, 1881, 2014]
[ 8.      0.     58.      0.6225  0.7289 10.7633]


C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 1
number of cluster center = 2237
...
order of connectivity matrix = 11
number of cluster center = 250
...
order of connectivity matrix = 21
number of cluster center = 79
...
order of connectivity matrix = 31
number of cluster center = 3
...
less then 4 cluster center
order 31
[144, 1881, 2014]
[8.000e+00 1.000e-04 3.100e+01 6.948e-01 8.332e-01 6.140e+00]
order of connectivity matrix = 1
number of cluster center = 2237
...


C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


less then 4 cluster center
order 10
[144, 1881, 2014]
[8.0000e+00 5.0000e-04 1.0000e+01 5.6310e-01 7.1740e-01 2.3547e+00]


In [26]:
epsilon_list = [0,1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
for epsilon in epsilon_list:
    print(np.round(Experiment(penbased_even_data,penbased_even_label,25,epsilon, 4000),4))


C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 1
number of cluster center = 4487
...
order of connectivity matrix = 11
number of cluster center = 25
...
order of connectivity matrix = 21
number of cluster center = 15
...
order of connectivity matrix = 31
number of cluster center = 12
...
order of connectivity matrix = 41
number of cluster center = 12
...
order of connectivity matrix = 51
number of cluster center = 11
...
order of connectivity matrix = 61
number of cluster center = 11
...
order of connectivity matrix = 71
number of cluster center = 10
...
order of connectivity matrix = 81
number of cluster center = 9
...
order of connectivity matrix = 91
number of cluster center = 9
...
order of connectivity matrix = 101
number of cluster center = 9
...
order of connectivity matrix = 111
number of cluster center = 9
...
order of connectivity matrix = 121
number of cluster center = 8
...
order of connectivity matrix = 131
number of cluster center = 8
...
order of connectivity matrix = 141
number of clus

order of connectivity matrix = 1211
number of cluster center = 6
...
order of connectivity matrix = 1221
number of cluster center = 6
...
order of connectivity matrix = 1231
number of cluster center = 6
...
order of connectivity matrix = 1241
number of cluster center = 6
...
order of connectivity matrix = 1251
number of cluster center = 6
...
order of connectivity matrix = 1261
number of cluster center = 6
...
order of connectivity matrix = 1271
number of cluster center = 6
...
order of connectivity matrix = 1281
number of cluster center = 6
...
order of connectivity matrix = 1291
number of cluster center = 6
...
order of connectivity matrix = 1301
number of cluster center = 6
...
order of connectivity matrix = 1311
number of cluster center = 6
...
order of connectivity matrix = 1321
number of cluster center = 6
...
order of connectivity matrix = 1331
number of cluster center = 6
...
order of connectivity matrix = 1341
number of cluster center = 6
...
order of connectivity matrix = 135

order of connectivity matrix = 2401
number of cluster center = 6
...
order of connectivity matrix = 2411
number of cluster center = 6
...
order of connectivity matrix = 2421
number of cluster center = 6
...
order of connectivity matrix = 2431
number of cluster center = 6
...
order of connectivity matrix = 2441
number of cluster center = 6
...
order of connectivity matrix = 2451
number of cluster center = 6
...
order of connectivity matrix = 2461
number of cluster center = 6
...
order of connectivity matrix = 2471
number of cluster center = 6
...
order of connectivity matrix = 2481
number of cluster center = 6
...
order of connectivity matrix = 2491
number of cluster center = 6
...
order of connectivity matrix = 2501
number of cluster center = 6
...
order of connectivity matrix = 2511
number of cluster center = 6
...
order of connectivity matrix = 2521
number of cluster center = 6
...
order of connectivity matrix = 2531
number of cluster center = 6
...
order of connectivity matrix = 254

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


...

order of connectivity matrix = 1
number of cluster center = 4487
...
order of connectivity matrix = 11
number of cluster center = 21
...
order of connectivity matrix = 21
number of cluster center = 13
...
order of connectivity matrix = 31
number of cluster center = 10
...
order of connectivity matrix = 41
number of cluster center = 6
...
order of connectivity matrix = 51
number of cluster center = 5
...
less then 4 cluster center
order 52
[285, 341, 649, 829]
[25.      0.     52.      0.9759  0.964  53.548 ]


C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 1
number of cluster center = 4487
...
order of connectivity matrix = 11
number of cluster center = 21
...
order of connectivity matrix = 21
number of cluster center = 6
...
order of connectivity matrix = 31
number of cluster center = 5
...
less then 4 cluster center
order 34
[285, 341, 649, 829]
[2.50000e+01 1.00000e-04 3.40000e+01 9.73600e-01 9.61100e-01 3.53997e+01]


C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 1
number of cluster center = 4487
...
order of connectivity matrix = 11
number of cluster center = 6
...
less then 4 cluster center
order 14
[285, 341, 649, 829]
[2.50000e+01 5.00000e-04 1.40000e+01 9.66000e-01 9.49800e-01 1.48169e+01]


In [19]:
epsilon_list = [0,1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
for epsilon in epsilon_list:
    print(np.round(Experiment(penbased_even_data,penbased_even_label,40,epsilon),4))


order of similarity matrix = 1
number of cluster center = 4487
...
order of similarity matrix = 11
number of cluster center = 7
...
order of similarity matrix = 21
number of cluster center = 5
...
less then 4 cluster center
order 28
[285, 649, 905, 3227]
[40.      0.     28.      0.84    0.8494]
order of similarity matrix = 1
number of cluster center = 4487
...
order of similarity matrix = 11
number of cluster center = 7
...
order of similarity matrix = 21
number of cluster center = 5
...
less then 4 cluster center
order 28
[285, 649, 905, 3227]
[40.      0.     28.      0.84    0.8494]
order of similarity matrix = 1
number of cluster center = 4487
...
order of similarity matrix = 11
number of cluster center = 7
...
order of similarity matrix = 21
number of cluster center = 5
...
less then 4 cluster center
order 28
[285, 649, 905, 3227]
[40.      0.     28.      0.84    0.8494]
order of similarity matrix = 1
number of cluster center = 4487
...
order of similarity matrix = 11
number of 

In [25]:
sigma_list = [3]
for sigma in sigma_list:
    print(np.round(Experiment(wine_data,wine_label,sigma,0),4))


order of similarity matrix = 1
number of cluster center = 178
...
order of similarity matrix = 501
number of cluster center = 109
...
[3.000e+00 0.000e+00 1.000e+03 2.370e-02 3.266e-01]


In [27]:
wine_data.describe()

,1,2,3,4,5,6,7,8,9,10,11,12,13
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


In [ ]:
from sklearn import preprocessing

x = wine_data.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
sigma_list = [0.1,0.2,0.3,0.4,0.5]
for sigma in sigma_list:
    print(np.round(Experiment(df,wine_label,sigma,0,3000),4))

In [ ]:
epsilon_list = [0,1e-3,5e-3,1e-2,5e-2,1e-4,5e-4]
for epsilon in epsilon_list:
    print(np.round(Experiment(df,wine_label,0.5,epsilon),4))


In [37]:
wdbc_data.describe()

,2,3,4,5,6,7,8,9,10,11,...,22,23,24,25,26,27,28,29,30,31
count,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,...,568.00000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000
mean,14.120491,19.305335,91.914754,654.279754,0.096321,0.104036,0.088427,0.048746,0.181055,0.062770,...,16.25315,25.691919,107.125053,878.578873,0.132316,0.253541,0.271414,0.114341,0.289776,0.083884
std,3.523416,4.288506,24.285848,351.923751,0.014046,0.052355,0.079294,0.038617,0.027319,0.007035,...,4.82232,6.141662,33.474687,567.846267,0.022818,0.156523,0.207989,0.065484,0.061508,0.018017
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.93000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.697500,16.177500,75.135000,420.175000,0.086290,0.064815,0.029540,0.020310,0.161900,0.057697,...,13.01000,21.095000,84.102500,514.975000,0.116600,0.146900,0.114475,0.064730,0.250350,0.071412
50%,13.355000,18.855000,86.210000,548.750000,0.095865,0.092525,0.061400,0.033455,0.179200,0.061515,...,14.96500,25.425000,97.655000,685.550000,0.131300,0.211850,0.226550,0.099840,0.282050,0.080015
75%,15.780000,21.802500,103.875000,782.625000,0.105300,0.130400,0.129650,0.073730,0.195625,0.066120,...,18.76750,29.757500,125.175000,1073.500000,0.146000,0.337600,0.381400,0.161325,0.317675,0.092065
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.04000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [78]:
x = wdbc_data.values #returns a numpy array
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
sigma_list = [0.1]
for sigma in sigma_list:
    print(np.round(Experiment(df,wdbc_label,sigma,0,1001),4))

order of similarity matrix = 1
number of cluster center = 568
...
order of similarity matrix = 501
number of cluster center = 204
...
order of similarity matrix = 1001
number of cluster center = 181
...
[1.000e-01 0.000e+00 1.001e+03 3.593e-01 2.779e-01]


In [79]:
epsilon_list = [0,1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
for epsilon in epsilon_list:
    print(np.round(Experiment(df,wdbc_label,0.1,epsilon,1001),4))

order of similarity matrix = 1
number of cluster center = 568
...
order of similarity matrix = 501
number of cluster center = 204
...
order of similarity matrix = 1001
number of cluster center = 181
...
[1.000e-01 0.000e+00 1.001e+03 3.593e-01 2.779e-01]
order of similarity matrix = 1
number of cluster center = 568
...
order of similarity matrix = 501
number of cluster center = 202
...
order of similarity matrix = 1001
number of cluster center = 181
...
[1.000e-01 0.000e+00 1.001e+03 3.587e-01 2.776e-01]
order of similarity matrix = 1
number of cluster center = 568
...
order of similarity matrix = 501
number of cluster center = 200
...
order of similarity matrix = 1001
number of cluster center = 167
...
[1.000e-01 0.000e+00 1.001e+03 2.004e-01 1.988e-01]
order of similarity matrix = 1
number of cluster center = 568
...
order of similarity matrix = 501
number of cluster center = 192
...
one cluster center
order 962
[73]
[1.00e-01 0.00e+00 9.62e+02 0.00e+00 0.00e+00]
order of similarity 

In [49]:
sigma_list = [50]
for sigma in sigma_list:
    print(np.round(Experiment(wdbc_data,wdbc_label,sigma,0,5001),4))


order of similarity matrix = 1
number of cluster center = 568
...
order of similarity matrix = 501
number of cluster center = 18
...
order of similarity matrix = 1001
number of cluster center = 17
...
order of similarity matrix = 1501
number of cluster center = 16
...
order of similarity matrix = 2001
number of cluster center = 16
...
order of similarity matrix = 2501
number of cluster center = 16
...
order of similarity matrix = 3001
number of cluster center = 15
...
order of similarity matrix = 3501
number of cluster center = 14
...
order of similarity matrix = 4001
number of cluster center = 13
...
order of similarity matrix = 4501
number of cluster center = 13
...
order of similarity matrix = 5001
number of cluster center = 13
...
[5.000e+01 0.000e+00 5.001e+03 5.340e-02 8.370e-02]


In [ ]:
epsilon_list = [0,1e-3,5e-3,1e-2,5e-2,1e-4,5e-4]
for epsilon in epsilon_list:
    print(np.round(Experiment(df,wine_label,20,epsilon,3000),4))

In [101]:
sigma_list = [32]
for sigma in sigma_list:
    print(np.round(Experiment(wine_data,wine_label,sigma,0,20000),4))


order of similarity matrix = 1
number of cluster center = 178
...
order of similarity matrix = 501
number of cluster center = 4
...
less then 3 cluster center
order 753
[18, 58, 86]
[3.200e+01 0.000e+00 7.530e+02 3.133e-01 4.279e-01]


In [27]:
epsilon_list = [0,1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
for epsilon in epsilon_list:
    print(np.round(Experiment(wine_data,wine_label,32,epsilon,10000),4))


C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 1
number of cluster center = 178
...
order of connectivity matrix = 11
number of cluster center = 12
...
order of connectivity matrix = 21
number of cluster center = 9
...
order of connectivity matrix = 31
number of cluster center = 7
...
order of connectivity matrix = 41
number of cluster center = 7
...
order of connectivity matrix = 51
number of cluster center = 7
...
order of connectivity matrix = 61
number of cluster center = 7
...
order of connectivity matrix = 71
number of cluster center = 6
...
order of connectivity matrix = 81
number of cluster center = 6
...
order of connectivity matrix = 91
number of cluster center = 5
...
order of connectivity matrix = 101
number of cluster center = 5
...
order of connectivity matrix = 111
number of cluster center = 5
...
order of connectivity matrix = 121
number of cluster center = 5
...
order of connectivity matrix = 131
number of cluster center = 5
...
order of connectivity matrix = 141
number of cluster cen

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 11
number of cluster center = 12
...
order of connectivity matrix = 21
number of cluster center = 9
...
order of connectivity matrix = 31
number of cluster center = 7
...
order of connectivity matrix = 41
number of cluster center = 7
...
order of connectivity matrix = 51
number of cluster center = 7
...
order of connectivity matrix = 61
number of cluster center = 7
...
order of connectivity matrix = 71
number of cluster center = 6
...
order of connectivity matrix = 81
number of cluster center = 6
...
order of connectivity matrix = 91
number of cluster center = 5
...
order of connectivity matrix = 101
number of cluster center = 5
...
order of connectivity matrix = 111
number of cluster center = 5
...
order of connectivity matrix = 121
number of cluster center = 5
...
order of connectivity matrix = 131
number of cluster center = 5
...
order of connectivity matrix = 141
number of cluster center = 5
...
order of connectivity matrix = 151
number of cluster cen

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


...

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])



number of cluster center = 5
...
order of connectivity matrix = 161
number of cluster center = 5
...
order of connectivity matrix = 171
number of cluster center = 5
...
order of connectivity matrix = 181
number of cluster center = 5
...
order of connectivity matrix = 191
number of cluster center = 5
...
order of connectivity matrix = 201
number of cluster center = 5
...
order of connectivity matrix = 211
number of cluster center = 5
...
order of connectivity matrix = 221
number of cluster center = 5
...
order of connectivity matrix = 231
number of cluster center = 5
...
order of connectivity matrix = 241
number of cluster center = 5
...
order of connectivity matrix = 251
number of cluster center = 5
...
order of connectivity matrix = 261
number of cluster center = 5
...
order of connectivity matrix = 271
number of cluster center = 4
...
order of connectivity matrix = 281
number of cluster center = 4
...
order of connectivity matrix = 291
number of cluster center = 4
...
order of conne

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])


order of connectivity matrix = 81
number of cluster center = 6
...
order of connectivity matrix = 91
number of cluster center = 5
...
order of connectivity matrix = 101
number of cluster center = 5
...
order of connectivity matrix = 111
number of cluster center = 5
...
order of connectivity matrix = 121
number of cluster center = 5
...
order of connectivity matrix = 131
number of cluster center = 5
...
order of connectivity matrix = 141
number of cluster center = 5
...
order of connectivity matrix = 151
number of cluster center = 5
...
order of connectivity matrix = 161
number of cluster center = 5
...
order of connectivity matrix = 171
number of cluster center = 5
...
order of connectivity matrix = 181
number of cluster center = 5
...
order of connectivity matrix = 191
number of cluster center = 5
...
order of connectivity matrix = 201
number of cluster center = 5
...
order of connectivity matrix = 211
number of cluster center = 5
...
order of connectivity matrix = 221
number of clust

C:\Users\jaemini_man\Desktop\GitHub\faster_CCE\CCE_algorithm.py:43: RuntimeWarning: invalid value encountered in double_scalars
  ncut = ncut + np.sum(P[label == l].T[label != l]) / len(label[label == l])
